In [1]:
#Needed???
!rm babies.sqlite

# Dependencies
import pandas as pd
import numpy as np
import os




In [2]:
csvfile = "../SSA_Baby_names_2.csv"

In [3]:
# Read CSV file into a pandas DataFrame
df = pd.read_csv(csvfile, dtype=object)

In [4]:
df.head()

,id,Decade,Males,num_boys,Females,num_girls
0,1,1880,John,"89,950",Mary,"91,668"
1,2,1880,William,"84,881",Anna,"38,159"
2,3,1880,James,"54,056",Emma,"25,404"
3,4,1880,George,"47,651",Elizabeth,"25,006"
4,5,1880,Charles,"46,656",Margaret,"21,799"


In [5]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [6]:
# Create an engine to a SQLite database file called `babies.sqlite`
engine = create_engine("sqlite:///babies.sqlite")

In [7]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [8]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Babies(Base):
    __tablename__ = 'babies'

    id = Column(Integer, primary_key=True)
    Decade = Column(Integer)
    Males = Column(Text)
    num_boys=Column(Integer)
    Females = Column(Text)
    num_girls=Column(Integer)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    #def __repr__(self):
    #    return '<Babies %r>' % (self.name)

In [9]:
# Use `create_all` to create the babies table in the database
Base.metadata.create_all(engine)

In [10]:
# Load the  csv file into a pandas dataframe
new_df = pd.read_csv(csvfile)
new_df[["Decade"]]=new_df[["Decade"]].apply(pd.to_numeric)
new_df.dtypes

id            int64
Decade        int64
Males        object
num_boys     object
Females      object
num_girls    object
dtype: object

In [11]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = new_df.to_dict(orient='records')

In [12]:
# Data is just a list of dictionaries that represent each row of data
print(data[:5])

[{'id': 1, 'Decade': 1880, 'Males': 'John', 'num_boys': '89,950', 'Females': 'Mary', 'num_girls': '91,668'}, {'id': 2, 'Decade': 1880, 'Males': 'William', 'num_boys': '84,881', 'Females': 'Anna', 'num_girls': '38,159'}, {'id': 3, 'Decade': 1880, 'Males': 'James', 'num_boys': '54,056', 'Females': 'Emma', 'num_girls': '25,404'}, {'id': 4, 'Decade': 1880, 'Males': 'George', 'num_boys': '47,651', 'Females': 'Elizabeth', 'num_girls': '25,006'}, {'id': 5, 'Decade': 1880, 'Males': 'Charles', 'num_boys': '46,656', 'Females': 'Margaret', 'num_girls': '21,799'}]


In [13]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [14]:
# Save the reference to the `babies` table as a variable called `table`
table = sqlalchemy.Table('babies', metadata, autoload=True)

In [15]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [16]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [17]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from babies limit 5").fetchall()

[(1, 1880, 'John', '89,950', 'Mary', '91,668'),
 (2, 1880, 'William', '84,881', 'Anna', '38,159'),
 (3, 1880, 'James', '54,056', 'Emma', '25,404'),
 (4, 1880, 'George', '47,651', 'Elizabeth', '25,006'),
 (5, 1880, 'Charles', '46,656', 'Margaret', '21,799')]